In [1]:
##by ltecho@whu.edu.cn
##
import pandas as pd
import numpy as np

In [2]:
#ip投放恶意样本数据，包含终端指纹，恶意样本名
ip_attack_file = pd.read_csv('ip_file\\ip_attack_file.csv',delimiter=',',encoding='utf-8')
#ip绑定过域名数据
ip_domain = pd.read_csv('ip_file\\ip_domain.csv',delimiter=',',encoding='utf-8')
#ip解析过域名数据
ip_log_dns_cate = pd.read_csv('ip_file\\ip_log_dns_cate.csv',delimiter=',',encoding='utf-8')

In [3]:
#整理ip终端恶意样本数据，按ip分组将多条记录数据组合，获取ip一一对应的终端指纹列表，投放过的恶意样本列表
ip_list = ip_attack_file['ip'].value_counts()
ip_list = pd.DataFrame(ip_list)
ip_list = ip_list.reset_index()
ip_list.rename(columns={'ip':'number','index':'ip_r'},inplace=True)
ip_list.rename(columns={'ip_r':'ip'},inplace=True)
ip_attack_file_cluster = []
file_name=np.repeat('',ip_list.shape[0])
ip_list['file_name']=file_name
mid_list=np.repeat('',ip_list.shape[0])
ip_list['mid_list']=mid_list
##获取ip，检测到的恶意文件名 列表，和终端指纹列表
for index,ip_df in ip_list.iterrows():
    ip_s = ip_df['ip']
    ip_attacks = ip_attack_file[ip_attack_file.ip==ip_s]['file_name'].value_counts()
    mid_attacks = ip_attack_file[ip_attack_file.ip==ip_s]['mid'].value_counts()
    ip_attacks = pd.DataFrame(ip_attacks)
    ip_attacks = ip_attacks.reset_index()
    mid_attacks = pd.DataFrame(mid_attacks)
    mid_attacks = mid_attacks.reset_index()
    file_name_set = set()
    mid_set = set()
    for index,mid_attack in mid_attacks.iterrows():
        mid_str = str(mid_attack['index'])
        mid_set.add(mid_str)
    for index,ip_attack in ip_attacks.iterrows():
        attack_str = str(ip_attack['index'])
        file_list = eval(attack_str)
        for k in range(0, len(file_list)):
            file_name_set.add(file_list[k].strip())
    ip_list.at[index,'file_name'] = file_name_set 
    ip_list.at[index,'mid_list'] = mid_set 
ip_list.to_csv('ip_file\\ip_attack_file_name.csv',index=0)

In [4]:
#处理ip绑定域名数据，按ip分组，从多条记录中提取出域名，组合成绑定过域名列表
ip_domain.drop(columns=['date','ip'],axis=1,inplace=True)
ip_domain.rename(columns={'source_ip':'ip'},inplace=True)
ip_domain.drop_duplicates(inplace=True)
ip_domain_cluster = ip_domain.groupby(['ip']).aggregate(lambda x:list(x))
ip_domain_cluster = ip_domain_cluster.reset_index()

In [5]:
#处理ip解析域名数据，按ip分组，从多条记录中提取出域名，组合成解析过域名列表
ip_log_dns_cate.drop(columns=['date'],axis=1,inplace=True)
ip_log_dns_cate.drop_duplicates(inplace=True)
ip_dns_cate_cluster = ip_log_dns_cate.groupby(['ip']).aggregate(lambda x:set(x))
ip_dns_cate_cluster = ip_dns_cate_cluster.reset_index()

In [7]:
#读取日志中存在的ip，作为其他数据筛选的依据
ip_data=pd.read_csv('ip_file\\result.csv',delimiter=',',encoding='utf-8')
#ip_data.drop(columns=['reason'],axis=1,inplace=True)
ip_data.rename(columns={'file_id':'ip'},inplace=True)

In [8]:
#集成以上数据，得到可做聚类分析的ip数据
ip_data_cluster = pd.merge(ip_data,ip_list,on='ip',how='left')
ip_data_cluster = pd.merge(ip_data_cluster,ip_domain_cluster,on='ip',how='left')
ip_data_cluster = pd.merge(ip_data_cluster,ip_dns_cate_cluster,on='ip',how='left')
ip_data_cluster.rename(columns={'number':'attack_file_number','file_name':'attack_file_list','domain_name':'resolve_domain_list','domain':'bind_domain_list'},inplace=True)
ip_data_cluster.drop(columns=['is_attacker','resolve_number'],axis=1,inplace=True)
ip_data_cluster.to_csv('ip_file\\ip_data_cluster.csv',index=0)

In [9]:
ip_data_cluster.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5879 entries, 0 to 5878
Data columns (total 8 columns):
ip                     5879 non-null object
reason                 4236 non-null object
attack_file_number     5642 non-null float64
attack_file_list       5642 non-null object
mid_list               5642 non-null object
bind_domain_list       143 non-null object
resolve_domain_list    4625 non-null object
domain_category        4625 non-null object
dtypes: float64(1), object(7)
memory usage: 413.4+ KB


In [ ]:
#ip_data_cluster = pd.read_csv('ip_file\\ip_data_cluster.csv',delimiter=',',encoding='utf-8')
###计算关联关系脚本，获得各个ip的关联ip列表
ip_data_cluster['mid_list'] = ip_data_cluster['mid_list'].astype(str)
ip_data_cluster['attack_file_list'] = ip_data_cluster['attack_file_list'].astype(str)
ip_data_cluster['bind_domain_list'] = ip_data_cluster['bind_domain_list'].astype(str)
ip_data_cluster['resolve_domain_list'] = ip_data_cluster['resolve_domain_list'].astype(str)
temp_list=np.repeat('',ip_data_cluster.shape[0])
ip_data_cluster['mid_friends_list']= temp_list
ip_data_cluster['attack_file_friends_list']= temp_list
ip_data_cluster['bind_domain_friends_list']= temp_list
ip_data_cluster['resolve_domain_friends_list']= temp_list
for index,ip_data in ip_data_cluster.iterrows():
    if ip_data.mid_list != 'nan' and ip_data.mid_list:
        print(ip_data.mid_list)
        mid_list = eval(ip_data.mid_list)
        mid_friends_list = set()
        print('one ip'+ip_data.ip)
        for k in mid_list:
            query_datas = ip_data_cluster.loc[ip_data_cluster['mid_list'].str.contains(k)]
            for index,query_data in query_datas.iterrows():
                if query_data.ip != ip_data.ip:
                    mid_friends_list.add(query_data.ip)
        if len(mid_friends_list)>0:
            ip_data_cluster.at[index,'mid_friends_list'] = mid_friends_list
        print(mid_friends_list)
    if ip_data.attack_file_list != 'nan' and ip_data.attack_file_list:
        attack_file_list = eval(ip_data.attack_file_list)
        attack_file_friends_list = set()
        #print(attack_file_list)
        for k in attack_file_list:
            #print(k)
            try:
                query_datas = ip_data_cluster.loc[ip_data_cluster['attack_file_list'].str.contains(str(k))]
                #print(query_datas)
                for index,query_data in query_datas.iterrows():
                    if query_data.ip != ip_data.ip:
                        attack_file_friends_list.add(query_data.ip)
            except:
                continue
        if len(attack_file_friends_list)>0:
            print('sucess')
            ip_data_cluster.at[index,'attack_file_friends_list'] = attack_file_friends_list 
        #print(attack_file_friends_list)
    if ip_data.bind_domain_list != 'nan' and ip_data.bind_domain_list:
        print(ip_data.bind_domain_list)
        bind_domain_list = eval(ip_data.bind_domain_list)
        bind_domain_friends_list = set()
        for k in bind_domain_list:
            query_datas = ip_data_cluster.loc[ip_data_cluster['bind_domain_list'].str.contains(str(k))]
            #print(query_datas)
            for index,query_data in query_datas.iterrows():
                if query_data.ip != ip_data.ip:
                    bind_domain_friends_list.add(query_data.ip)
        if len(bind_domain_friends_list)>0:
            ip_data_cluster.at[index,'bind_domain_friends_list'] = bind_domain_friends_list
        #print(bind_domain_friends_list)
#     if ip_data.resolve_domain_list != 'nan':
#         resolve_domain_list = eval(ip_data.resolve_domain_list)
#         #print(ip_data.resolve_domain_list)
#         resolve_domain_friends_list = set()
#         for k in resolve_domain_list:
#             print(k)
#             query_datas = ip_data_cluster.loc[ip_data_cluster['resolve_domain_list'].str.contains(k)]
#             for index,query_data in query_datas.iterrows():
#                 if query_data.ip != ip_data.ip:
#                     resolve_domain_friends_list.add(query_data.ip)
#         if len(resolve_domain_friends_list)>0:
#             ip_data_cluster.at[index,'resolve_domain_friends_list'] = resolve_domain_friends_list
        #print(resolve_domain_friends_list)
    
ip_data_cluster.to_csv('ip_data_friends.csv',index=0)